In [4]:
from pathlib import Path
import pandas as pd
import re
ROOT = Path('../..')
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/housing')

In [5]:
# Set the path to the raw data
p = Path('../../raw/properties')

# Define a pattern to match the year in the filename
pattern = r'\d{4}'
columns=['geography_code', 'geography_name']
dates = []
# Define an empty dataframe to append the results to
properties = pd.DataFrame(columns=columns)
# Iterate through the proprties directory
for root, dirs, files in p.walk():

    # Resolve the path
    DDIR = Path(root).resolve()

    # Go through each file name in the root directory
    for file in files:

        # Match the date
        date = re.search(pattern, file).group()
        if date:
            dates.append(date)
            # Read the csv
            d = pd.read_csv(DDIR / file, na_values=['..', '-'])

            # Select Local auhtorities, met counties, regions and national values only.
            d = d.loc[d['geography'].isin(['NATL','REGL','LAUA','CTYMET'])]

            # Slice the frame
            d = d[['ecode', 'area_name', 'all_properties']]

            # Rename columns 
            d.rename(columns={'ecode': 'geography_code', 'area_name': 'geography_name', 'all_properties': f'{date}'}, inplace=True)
            d = d.melt(id_vars=columns, value_vars=f'{date}', var_name='date')
        else:
            print('No date found')
        # properties = properties.merge(d, on=['geography_code', 'geography_name'], how='outer') 
        properties = pd.concat([properties, d])

In [6]:
# Reorder by date desc
properties.sort_values(by='date', inplace=True)

# Make geo code the index
properties.set_index('geography_code', inplace=True)

# Write to parquet
properties.to_parquet(ROOT / 'data/properties/site/all_properties.parquet')

# properties.to_csv(ROOT / 'data/properties/standard/all_properties.csv')